In [10]:
import pandas as pd
import numpy as np
from faker import Faker
import random

# Initialize Faker for data generation
fake = Faker()
Faker.seed(42)
random.seed(42)
np.random.seed(42)

# Define dataset size
NUM_CUSTOMERS = 10000
NUM_BRANCHES = 50  # One bank with 50 branches

# Generate Bank Branches (One bank with multiple branches)
branches = []
for i in range(1, NUM_BRANCHES + 1):
    city = fake.city()
    state = fake.state_abbr()
    zip_code = fake.zipcode()
    branches.append([i, "USA National Bank", city, state, zip_code])

branches_df = pd.DataFrame(branches, columns=["BranchID", "BankName", "City", "State", "ZipCode"])

# Generate Customers Table with Addresses and Zip Codes
customers = []
customer_branch_map = {}  # Track assigned branches per customer

for i in range(1, NUM_CUSTOMERS + 1):
    credit_score = random.randint(300, 850)
    income_level = random.choice(["Low", "Medium", "High"])
    if income_level == "Low":
        account_balance = round(random.uniform(1000, 10000), 2)
    elif income_level == "Medium":
        account_balance = round(random.uniform(10000, 50000), 2)
    else:
        account_balance = round(random.uniform(50000, 150000), 2)

    # Assign customer to a branch in the same area (city, state, zip)
    branch = random.choice(branches)  # Select a branch
    customer_branch_map[i] = branch[0]  # Store branch ID for this customer

    customers.append([
        i,  # CustomerID
        fake.name(),
        random.randint(18, 80),  # Age
        income_level,
        credit_score,
        random.choice(["Yes", "No"]),  # LoanHistory
        account_balance,
        fake.street_address(),  # Street Address
        branch[2],  # City (Matches branch location)
        branch[3],  # State (Matches branch location)
        branch[4],  # Zip Code (Matches branch location)
        branch[0]  # BranchID (FK)
    ])

customers_df = pd.DataFrame(customers, columns=[
    "CustomerID", "Name", "Age", "IncomeLevel", "CreditScore", "LoanHistory", "AccountBalance", 
    "StreetAddress", "City", "State", "ZipCode", "BranchID"
])

# Save Customers and Branches Tables
customers_df.to_excel("customers.xlsx", index=False)
branches_df.to_excel("branches.xlsx", index=False)

print("Customers dataset with Zip Codes has been generated and saved as 'customers.xlsx'!")


✅ Customers dataset with Zip Codes has been generated and saved as 'customers.xlsx'!


In [11]:
import pandas as pd
import numpy as np
from faker import Faker
import random

# Initialize Faker for data generation
fake = Faker()
Faker.seed(42)
random.seed(42)
np.random.seed(42)

# Load previously created Customers and Branches tables
customers_df = pd.read_excel("customers.xlsx")
branches_df = pd.read_excel("branches.xlsx")

# Define dataset size
NUM_CARDS = 15000  # More cards than customers (some have multiple cards)

# Generate CreditCards Table (Ensuring each card belongs to a valid CustomerID and their assigned BranchID)
cards = []
card_id_counter = 1
customer_card_map = {}  # Track assigned cards per customer

for customer_id in customers_df["CustomerID"]:
    num_cards = random.randint(1, 3)  # Each customer has 1 to 3 credit cards
    customer_card_map[customer_id] = []

    for _ in range(num_cards):
        card_id = card_id_counter
        card_id_counter += 1
        expiry_date = fake.date_between(start_date="+1y", end_date="+10y")

        # Ensure card is linked to the same Branch as the Customer
        branch_id = customers_df.loc[customers_df["CustomerID"] == customer_id, "BranchID"].values[0]

        cards.append([
            card_id,  # CardID
            customer_id,  # CustomerID (FK)
            branch_id,  # BranchID (FK)
            random.choice(["Visa", "MasterCard", "Amex", "Discover"]),
            random.randint(1000, 20000),  # CreditLimit
            expiry_date
        ])
        customer_card_map[customer_id].append(card_id)

cards_df = pd.DataFrame(cards, columns=["CardID", "CustomerID", "BranchID", "CardType", "CreditLimit", "ExpiryDate"])

# Save CreditCards Table
cards_df.to_excel("credit_cards.xlsx", index=False)

print("CreditCards dataset has been generated and saved as 'credit_cards.xlsx'!")


✅ CreditCards dataset has been generated and saved as 'credit_cards.xlsx'!


In [12]:
import pandas as pd
import numpy as np
from faker import Faker
import random

# Initialize Faker for data generation
fake = Faker()
Faker.seed(42)
random.seed(42)
np.random.seed(42)

# Load previously created Branches table
branches_df = pd.read_excel("branches.xlsx")

# Define dataset size
NUM_MERCHANTS = 2000

# Generate Merchants Table (Matching locations with existing Branches)
merchants = []
merchant_category_list = ["Retail", "Online Shopping", "Gas Station", "Travel", 
                          "Entertainment", "Groceries", "Electronics", "Luxury Goods"]

for merchant_id in range(1, NUM_MERCHANTS + 1):
    branch = branches_df.sample(n=1).iloc[0]  # Assign merchant to an existing branch location
    merchants.append([
        merchant_id,  # MerchantID
        fake.company(),  # Merchant Name
        random.choice(merchant_category_list),  # Category
        branch["City"],  # City (Matches branch location)
        branch["State"],  # State (Matches branch location)
        branch["ZipCode"],  # Zip Code (Matches branch location)
        branch["BranchID"]  # BranchID (FK) - Links merchant to branch location
    ])

merchants_df = pd.DataFrame(merchants, columns=[
    "MerchantID", "MerchantName", "Category", "City", "State", "ZipCode", "BranchID"
])

# Save Merchants Table
merchants_df.to_excel("merchants.xlsx", index=False)

print("Merchants dataset has been generated and saved as 'merchants.xlsx'!")


✅ Merchants dataset has been generated and saved as 'merchants.xlsx'!


In [13]:
import pandas as pd

# Generate Fraud Rules Table

# Define fraud categories with explanations
fraud_types = [
    [1, "No Fraud", "Transaction appears normal."],
    [2, "Large Transaction Size", "Transaction amount exceeds 70% of the card's credit limit."],
    [3, "High Transaction Frequency", "Multiple transactions within a short time period."],
    [4, "High-Risk Merchant", "Transaction at high-risk category merchants (Luxury, Electronics, Travel)."],
    [5, "Geo-Location Anomaly", "Back-to-back transactions from different states or distant locations."],
    [6, "Card Not Present Transaction", "Online transaction without prior history of similar purchases."],
    [7, "Credit Score Mismatch", "Low-credit-score customer making unusually large transactions."]
]

fraud_types_df = pd.DataFrame(fraud_types, columns=["FraudTypeID", "Description", "RuleExplanation"])

# Save Fraud Rules Table
fraud_types_df.to_excel("fraud_rules.xlsx", index=False)

print("Fraud Rules dataset has been generated and saved as 'fraud_rules.xlsx'!")


✅ Fraud Rules dataset has been generated and saved as 'fraud_rules.xlsx'!


In [14]:
import pandas as pd
import random

# Load previously created Transactions table to reference valid TransactionIDs
transactions_df = pd.read_excel("transactions.xlsx")

# Define dataset size
NUM_CHARGEBACKS = 1000

# Ensure we don't exceed available transactions
chargeback_transaction_ids = transactions_df.sample(n=NUM_CHARGEBACKS)["TransactionID"].tolist()

# Generate Chargebacks Table
chargebacks = []
dispute_reasons = ["Unauthorized transaction", "Duplicate charge", "Product not received", "Service not as described"]
resolution_statuses = ["Approved", "Denied", "Pending"]

for chargeback_id, transaction_id in enumerate(chargeback_transaction_ids, start=1):
    chargebacks.append([
        chargeback_id,  # ChargebackID
        transaction_id,  # TransactionID (FK)
        random.choice(dispute_reasons),  # Dispute Reason
        random.choice(resolution_statuses)  # Resolution Status
    ])

chargebacks_df = pd.DataFrame(chargebacks, columns=["ChargebackID", "TransactionID", "DisputeReason", "ResolutionStatus"])

# Save Chargebacks Table
chargebacks_df.to_excel("chargebacks.xlsx", index=False)

print("Chargebacks dataset has been generated and saved as 'chargebacks.xlsx'!")


FileNotFoundError: [Errno 2] No such file or directory: 'transactions.xlsx'

In [15]:
import pandas as pd

# Generate Transaction Types Table

# Define transaction types
transaction_types = [
    [1, "POS Purchase"],
    [2, "Online Purchase"],
    [3, "ATM Withdrawal"],
    [4, "Bill Payment"],
    [5, "Bank Transfer"]
]

# Create DataFrame
transaction_types_df = pd.DataFrame(transaction_types, columns=["TypeID", "Description"])

# Save Transaction Types Table
transaction_types_df.to_excel("transaction_types.xlsx", index=False)

print("Transaction Types dataset has been generated and saved as 'transaction_types.xlsx'!")


✅ Transaction Types dataset has been generated and saved as 'transaction_types.xlsx'!


In [3]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime

# Initialize Faker for data generation
fake = Faker()
Faker.seed(42)
random.seed(42)
np.random.seed(42)

# Load required tables
customers_df = pd.read_excel("customers.xlsx")
cards_df = pd.read_excel("credit_cards.xlsx")
merchants_df = pd.read_excel("merchants.xlsx")
transaction_types_df = pd.read_excel("transaction_types.xlsx")
fraud_rules_df = pd.read_excel("fraud_rules.xlsx")

# Define dataset size
NUM_TRANSACTIONS = 100000

# Generate Transactions Table
transactions = []
start_date = datetime(2022, 1, 1)
end_date = datetime(2025, 3, 31)

for transaction_id in range(1, NUM_TRANSACTIONS + 1):
    customer = customers_df.sample(n=1).iloc[0]  # Select a random customer
    customer_id = customer["CustomerID"]
    branch_id = customer["BranchID"]  # Get customer's branch

    # Select a credit card that belongs to the chosen customer
    customer_cards = cards_df[cards_df["CustomerID"] == customer_id]
    if customer_cards.empty:
        continue  # Skip if no card is found

    card = customer_cards.sample(n=1).iloc[0]
    card_id = card["CardID"]

    # Select a merchant from the same region as the customer
    merchants_in_region = merchants_df[
        (merchants_df["City"] == customer["City"]) & 
        (merchants_df["State"] == customer["State"])
    ]
    if merchants_in_region.empty:
        continue  # Skip if no merchant is found in the region

    merchant = merchants_in_region.sample(n=1).iloc[0]
    merchant_id = merchant["MerchantID"]

    # Random transaction details
    amount = round(random.uniform(5, 10000), 2)
    type_id = random.choice(transaction_types_df["TypeID"].tolist())
    timestamp = fake.date_time_between(start_date=start_date, end_date=end_date)
    fraud_type_id = 1  # Default to "No Fraud"

    # Apply fraud detection logic based on fraud rules
    credit_limit = card["CreditLimit"]
    customer_credit_score = customer["CreditScore"]

    if amount > 0.7 * credit_limit:
        fraud_type_id = 2  # Large Transaction Size
    elif random.random() < 0.005:
        fraud_type_id = 3  # High Transaction Frequency
    elif merchant["Category"] in ["Luxury Goods", "Electronics", "Travel"]:
        fraud_type_id = 4  # High-Risk Merchant
    elif random.random() < 0.0025:
        fraud_type_id = 5  # Geo-Location Anomaly
    elif type_id == 2 and random.random() < 0.01:
        fraud_type_id = 6  # Card Not Present Transaction
    elif customer_credit_score < 500 and amount > 0.3 * credit_limit:
        fraud_type_id = 7  # Credit Score Mismatch

    transactions.append([
        transaction_id,
        customer_id,
        card_id,
        merchant_id,
        amount,
        type_id,
        timestamp,
        fraud_type_id,
        branch_id  # Include branch ID
    ])

transactions_df = pd.DataFrame(transactions, columns=[
    "TransactionID", "CustomerID", "CardID", "MerchantID", "Amount",
    "TypeID", "Timestamp", "FraudTypeID", "BranchID"
])

# Save Transactions Table
transactions_df.to_excel("transactions.xlsx", index=False)

print("Transactions dataset has been generated and saved as 'transactions.xlsx'!")


✅ Transactions dataset has been generated and saved as 'transactions.xlsx'!


In [17]:
import pandas as pd
import random

# Load Transactions table to reference valid TransactionIDs
transactions_df = pd.read_excel("transactions.xlsx")

# Define dataset size (subset of transactions)
NUM_CHARGEBACKS = 1000

# Ensure we don't exceed available transactions
chargeback_transaction_ids = transactions_df.sample(n=NUM_CHARGEBACKS)["TransactionID"].tolist()

# Generate Chargebacks Table
chargebacks = []
dispute_reasons = ["Unauthorized transaction", "Duplicate charge", "Product not received", "Service not as described"]
resolution_statuses = ["Approved", "Denied", "Pending"]

for chargeback_id, transaction_id in enumerate(chargeback_transaction_ids, start=1):
    chargebacks.append([
        chargeback_id,  # ChargebackID
        transaction_id,  # TransactionID (FK)
        random.choice(dispute_reasons),  # Dispute Reason
        random.choice(resolution_statuses)  # Resolution Status
    ])

chargebacks_df = pd.DataFrame(chargebacks, columns=["ChargebackID", "TransactionID", "DisputeReason", "ResolutionStatus"])

# Save Chargebacks Table
chargebacks_df.to_excel("chargebacks.xlsx", index=False)

print("Chargebacks dataset has been generated and saved as 'chargebacks.xlsx'!")


✅ Chargebacks dataset has been generated and saved as 'chargebacks.xlsx'!
